In [20]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score

In [140]:
# Читаем
# Там разделитель \t и нет имен столбцов
data_train = pd.read_csv('train.txt', delimiter='\t', header=0, names=['name', 'group'])
data_validate = pd.read_csv('test.txt', delimiter='\t', header=0, names=['name'])

# В нижний регистр
data_train['name'] = data_train['name'].str.lower()
#data_train['group'] = data_train['group'].str.lower()

data_validate['name'] = data_validate['name'].str.lower()

data_train

,name,group
0,державка 30531402 mapal,Резцы
1,"кабель кпсввнг-ls 1х2х0,75",Кабельная продукция
2,"трубка электроизоляционная ткр ф16,0мм",Изделия электроустан
3,"лента конвейер 2,1-1000-тк-200-2-5/2",ИзделияРезино-технич
4,соединение быстроразъемное hs-ss-e-0100,Запчасти
...,...,...
23967,"фреза шпоночная ц/х 8,0",Фрезы
23968,кирпич керам полнотел одинарный м200,МатерСтроительные
23969,"клеймо тв спл 122""ф"" вк15",Инструменты
23970,элемент питания saft ls 14250/std 1/2aa,Запчасти


In [141]:
# Пустых значений там нет. А какие есть?
group_counts = data_train['group'].value_counts()
group_counts

Запчасти                5757
Метизы                  1378
З/Ч АвтомобПромышл      1257
З/Ч по чертежам          985
Инструменты              826
                        ... 
ПродукцЦеллюлозБумаж      39
Пилы                      37
ЗаготовкаИнстр и з/ч      34
Цепи и звенья             34
Теплоизоляционные         13
Name: group, Length: 96, dtype: int64

In [142]:
# Удаляем всё, что не буквы и не цифры
#data_train['name'] = data_train['name'].str.replace('[^a-zA-Zа-яА-Я1-9 ]', ' ')
#data_validate['name'] = data_validate['name'].str.replace('[^a-zA-Zа-яА-Я1-9 ]', ' ')
data_train['name'] = data_train['name'].str.replace('[^a-zA-Zа-яА-Я]', ' ')
data_validate['name'] = data_validate['name'].str.replace('[^a-zA-Zа-яА-Я]', ' ')

data_train

,name,group
0,державка mapal,Резцы
1,кабель кпсввнг ls х х,Кабельная продукция
2,трубка электроизоляционная ткр ф мм,Изделия электроустан
3,лента конвейер тк,ИзделияРезино-технич
4,соединение быстроразъемное hs ss e,Запчасти
...,...,...
23967,фреза шпоночная ц х,Фрезы
23968,кирпич керам полнотел одинарный м,МатерСтроительные
23969,клеймо тв спл ф вк,Инструменты
23970,элемент питания saft ls std aa,Запчасти


In [143]:
# Удаляем слова короче определенного количества символов

min_len = 2
data_train['name'] = data_train['name'].str.split().map(lambda sl: " ".join(s for s in sl if len(s) >= min_len))
data_validate['name'] = data_validate['name'].str.split().map(lambda sl: " ".join(s for s in sl if len(s) >= min_len))

data_train

,name,group
0,державка mapal,Резцы
1,кабель кпсввнг ls,Кабельная продукция
2,трубка электроизоляционная ткр мм,Изделия электроустан
3,лента конвейер тк,ИзделияРезино-технич
4,соединение быстроразъемное hs ss,Запчасти
...,...,...
23967,фреза шпоночная,Фрезы
23968,кирпич керам полнотел одинарный,МатерСтроительные
23969,клеймо тв спл вк,Инструменты
23970,элемент питания saft ls std aa,Запчасти


In [144]:
# Проверяем, не появилось ли пустых строк. Спойлер - появились
print(data_train[data_train['name'] == ''])
print('---------------------------------------------')
print(data_validate[data_validate['name'] == ''])

Empty DataFrame
Columns: [name, group]
Index: []
---------------------------------------------
Empty DataFrame
Columns: [name]
Index: []


In [145]:
# Заменим категории на onehot
#!!! Это можно не делать, как видно ниже
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_train_full = enc.fit_transform(data_train['group'].to_numpy().reshape(-1, 1)).toarray()
y_train_full

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [146]:
# Сделаем baseline обычной векторизацией
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
X_train_full = count_vectorizer.fit_transform(data_train['name'])
X_validate_full = count_vectorizer.transform(data_validate['name'])

X_train_full

<23972x11128 sparse matrix of type '<class 'numpy.int64'>'
	with 75162 stored elements in Compressed Sparse Row format>

In [147]:
# Еще попробуем tf_idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_train_full = tfidf_vectorizer.fit_transform(data_train['name'])
X_validate_full = tfidf_vectorizer.transform(data_validate['name'])

X_train_full

<23972x11128 sparse matrix of type '<class 'numpy.float64'>'
	with 75162 stored elements in Compressed Sparse Row format>

In [148]:
# Разделение на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train_full, data_train['group'], test_size=0.2, random_state=42)

In [149]:
# Юзнем случайный лес
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(max_depth=100, random_state=42, n_jobs=3)
rf_clf.fit(X_train, y_train)

y_pred = rf_clf.predict(X_test)
# Корректно работает для двумерных массивов
print('rf_clf accuracy score: ', accuracy_score(y_test, y_pred))

rf_clf accuracy score:  0.6588112617309697


In [150]:
# Юзнем логистическую регрессию
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(X_train, y_train)

y_pred = lr_clf.predict(X_test)
# Корректно работает для двумерных массивов
print('lr_clf accuracy score: ', accuracy_score(y_test, y_pred))


lr_clf accuracy score:  0.7023983315954119


c:\users\boget\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [151]:
# Юзнем SGD
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3)
sgd_clf.fit(X_train, y_train)

y_pred = sgd_clf.predict(X_test)
# Корректно работает для двумерных массивов
print('sgd_clf accuracy score: ', accuracy_score(y_test, y_pred))

sgd_clf accuracy score:  0.764754953076121


In [136]:
# Юзнем SVM
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

svm_clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
#svm_clf = SVC(gamma='auto')
svm_clf.fit(X_train, y_train)

y_pred = svm_clf.predict(X_test)
# Корректно работает для двумерных массивов
print('svm_clf accuracy score: ', accuracy_score(y_test, y_pred))

svm_clf accuracy score:  0.6617309697601669
